# RAISERROR syntax

Given this `RAISERROR` statement

`RAISERROR('You cannot apply a 50%% discount on %s number %d', 6, 1, 'product', 5);`

Which of the following outputs will you get if you execute this code?

- "You cannot apply a 50% discount on product number 5"

# CATCHING the RAISERROR

You need to select a product from the `products` table using a given `product_id`.

If the select statement doesn't find any product, you want to raise an error using the `RAISERROR` statement. You also need to catch possible errors in the execution.

```
BEGIN TRY
	-- Change the value
    DECLARE @product_id INT = 5;	
    IF NOT EXISTS (SELECT * FROM products WHERE product_id = @product_id)
        RAISERROR('No product with id %d.', 11, 1, @product_id);
    ELSE 
        SELECT * FROM products WHERE product_id = @product_id;
END TRY
BEGIN CATCH
	SELECT ERROR_MESSAGE();
END CATCH
```

# THROW with or without parameters

Which of the following is true about the `THROW` statement?

- The `THROW` statement without parameters should be placed within a `CATCH` block.

# THROW without parameters

You want to prepare a stored procedure to insert new products in the database. In that stored procedure, you want to insert the possible errors in a table called `errors`, and after that, re-throw the original error.

How do you prepare the stored procedure?

```
CREATE PROCEDURE insert_product
  @product_name VARCHAR(50),
  @stock INT,
  @price DECIMAL

AS

BEGIN TRY
	INSERT INTO products (product_name, stock, price)
		VALUES (@product_name, @stock, @price);
END TRY
-- Set up the CATCH block
BEGIN CATCH
	-- Insert the error and end the statement with a semicolon
    INSERT INTO errors VALUES ('Error inserting a product');
    -- Re-throw the error
	THROW; 
END CATCH
```

# Executing a stored procedure that throws an error

You want to register that you received 3 Trek Conduit+ bikes with a price of $499.99. You think Trek Conduit+ doesn't exist in the `products` table, so you try to insert it as a new product, using the stored procedure you created in the previous exercise:
```
CREATE PROCEDURE insert_product
  @product_name VARCHAR(50),
  @stock INT,
  @price DECIMAL

AS

BEGIN TRY
    INSERT INTO products (product_name, stock, price)
        VALUES (@product_name, @stock, @price);
END TRY
BEGIN CATCH    
    INSERT INTO errors VALUES ('Error inserting a product');  
    THROW;  
END CATCH
```
You need to catch the possible errors generated in the execution of the stored procedure, showing the original error message.

How do you prepare the script?

```
BEGIN TRY
	-- Execute the stored procedure
	EXEC insert_product
    	-- Set the values for the parameters
    	@product_name = 'Trek Conduit+',
        @stock = 3,
        @price = 499.99;
END TRY
-- Set up the CATCH block
BEGIN CATCH
	-- Select the error message
	SELECT ERROR_MESSAGE();
END CATCH
```

# THROW with parameters

You need to prepare a script to select all the information of a member from the `staff` table using a given `staff_id`.

If the select statement doesn't find any member, you want to throw an error using the `THROW` statement. You need to warn there is no staff member with such id.

```
DECLARE @staff_id INT = 4;

IF NOT EXISTS (SELECT * FROM staff WHERE staff_id = @staff_id)
   	-- Invoke the THROW statement with parameters
	THROW 50001, 'No staff member with such id', 1;
ELSE
   	SELECT * FROM staff WHERE staff_id = @staff_id
```

# Ways of customizing error messages

You want to use the `THROW` statement to throw an error with a custom message. Which of the following is a possible option to do so?

- You use the `FORMATMESSAGE` function and save the result into a variable that you pass to the `THROW` statement.

# Concatenating the message

You need to prepare a script to select all the information about the members from the `staff` table using a given `first_name`.

If the select statement doesn't find any member, you want to throw an error using the `THROW` statement. You need to warn there is no staff member with such a name.

```
DECLARE @first_name NVARCHAR(20) = 'Pedro';

-- Concat the message
DECLARE @my_message NVARCHAR(500) =
	CONCAT('There is no staff member with ', @first_name, ' as the first name.');

IF NOT EXISTS (SELECT * FROM staff WHERE first_name = @first_name)
	-- Throw the error
	THROW 50000, @my_message, 1;
```

# FORMATMESSAGE with message string

Every time you sell a bike in your store, you need to check if there is enough stock. You prepare a script to check it and throw an error if there is not enough stock.

```
DECLARE @product_name AS NVARCHAR(50) = 'Trek CrossRip+ - 2018';
-- Set the number of sold bikes
DECLARE @sold_bikes AS INT = 10;
DECLARE @current_stock INT;

SELECT @current_stock = stock FROM products WHERE product_name = @product_name;

DECLARE @my_message NVARCHAR(500) =
	-- Customize the error message
	FORMATMESSAGE('There are not enough %s bikes. You have %d in stock.' , @product_name, @current_stock);

IF (@current_stock - @sold_bikes < 0)
	-- Throw the error
	THROW 50000, @my_message, 1;
```

# FORMATMESSAGE with message number

Like in the previous exercise, you need to check if there is enough stock when you sell a product.

This time you want to add your custom error message to the `sys.messages` catalog, by executing the `sp_addmessage` stored procedure. Once you have added the message, you can use it in the `FORMATMESSAGE` function.

```
EXEC sp_addmessage @msgnum = 50002, @severity = 16, @msgtext = 'There are not enough %s bikes. You only have %d in stock.', @lang = N'us_english';

DECLARE @product_name AS NVARCHAR(50) = 'Trek CrossRip+ - 2018';
DECLARE @sold_bikes AS INT = 10;
DECLARE @current_stock INT;

SELECT @current_stock = stock FROM products WHERE product_name = @product_name;

DECLARE @my_message NVARCHAR(500) =
	-- Prepare the error message
	FORMATMESSAGE(50002, @product_name, @current_stock);

IF (@current_stock - @sold_bikes < 0)
	-- Throw the error
	THROW 50000, @my_message, 1;
```